In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
import time
from pyirf import simulations,utils, spectral




In [ ]:
2+2

In [ ]:
int_cut = 0.3
min_pixel = 70

# MuonIntensityFitter : 
#   hole_radius_m : [["type", "LST_*", 0.308]]

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})


muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.3"],
                                                        ["ring_completeness", "parameters.completeness > 0.3"]]},
                                    "ImageParameterQuery" : {
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]},
                                    "MuonIntensityFitter" : {
                                        "hole_radius_m" : [["type", "LST_*", 0.308]]
                                    }}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()
event_ids = []
impacts = []
widths = []
for k in range(1,101):
    filename = f'/Users/vdk/LST/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    print(filename)
    source = EventSource(filename)
    event_iterator = iter(source)
    keks = k*1000
    checker = 0
    for i,event in enumerate(event_iterator):
        for m in event.trigger.tels_with_trigger:
            checker += 1
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
            #muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            print(f"Non Muon Event Telescope with stereo trigger = {event.trigger.tels_with_trigger}")
            calib(event)
            image_processor(event)
            muon_processor(event)
            for m in event.muon.tel.keys():
                if not np.isnan(event.muon.tel[m].efficiency.width):
                    #print("I am here")
                    # plt.figure(figsize = (9,6))
                    # camgeom = source.subarray.tel[1].camera.geometry
                    # title = f'event {event.index.event_id} in telescope {m}'
                    # disp = CameraDisplay(camgeom,title=title)
                    # #disp.image = event.r1.tel[m].waveform.sum(axis = 1)
                    # disp.image = event.dl1.tel[m].image
                    # disp.cmap = plt.cm.Reds
                    # disp.add_colorbar()
                    # disp.set_limits_percent(95)
                    # plt.show()
                    print(f"Muon Event Telescope with stereo trigger = {event.trigger.tels_with_trigger}")
                    event_container[keks+i] = event
                    event_ids.append(event.index.event_id)
                    impacts.append(event.muon.tel[m].efficiency.impact)
                    widths.append(event.muon.tel[m].efficiency.width)
    t2 = time.perf_counter(), time.process_time()
    print("checker = ", checker)   
#plt.close()
print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:

unusable_pixels_hg = event.mon.tel[tel_id].calibration.unusable_pixels[0]
unusable_pixels_lg = event.mon.tel[tel_id].calibration.unusable_pixels[1]

bad_pixels = unusable_pixels_hg | unusable_pixels_lg

event.dl1.tel[tel_id] *= ~bad_pixels


In [ ]:
for items in event_container[1003].r1.tel.items():
    print()

In [ ]:
event_container[1003].r1.tel[2].waveform.shape

In [ ]:
mask = np.array([True,True,False])
bad_pixels = test*[~mask]
test = np.array([1,2,3])

In [ ]:
test *= ~mask
test

In [ ]:
#print("all events = ", all_events)
#print("stereo = ", stereo)
print("muons = ", len(event_container))
print("muons events = ", event_container.keys())
#for i in event_container_protons:
#    print(i)

opt_eff_m = []
for i in event_container:
    for m in event_container[i].trigger.tels_with_trigger:
        if not np.isnan(event_container[i].muon.tel[m].efficiency.optical_efficiency) and event_container[i].muon.tel[m].efficiency.optical_efficiency < 100:
            opt_eff_m.append(event_container[i].muon.tel[m].efficiency.optical_efficiency)

print("mean = ", np.mean(opt_eff_m))
print("std = ", np.std(opt_eff_m))
print("len = ", len(opt_eff_m))
print(f"event_ids = {event_ids}")


In [ ]:
plt.hist(widths, bins = 12, alpha = 0.7)
plt.ylabel('Counts')
plt.xlabel('Ring widths in deg')
plt.xlim(0,0.12)
plt.ylim(0,90)
plt.grid(alpha = 0.5)
plt.title("Widths of muons processed by CalibPipe method")

In [ ]:
plt.hist(impacts, bins = 10, alpha = 0.7)
plt.ylabel('Counts')
plt.xlabel('Ring impact in m')
plt.xlim(0,12)
plt.ylim(0,90)
plt.grid(alpha = 0.5)
plt.title("Impact of muons processed by CalibPipe method")

In [ ]:
energies = []
for k in range(1,100):
    filename = f'/Users/vdk/LST/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    keks = k*1000
    for i,event in enumerate(event_iterator):
        energies.append(event.simulation.shower.energy.to_value())

In [ ]:
gev_energies = np.array(energies)
histplot(gev_energies*1000, log_scale = True, bins = 15)
plt.xlabel("Energy in GeV")
plt.title("Muon Energies")

In [ ]:
w_energies = [energy**(-0.7) for energy in energies]
sum(w_energies)

In [ ]:
simulated_spectrum = spectral.PowerLaw(normalization = 1 * spectral.DIFFUSE_FLUX_UNIT, index = -2, e_ref =1* u.TeV)
target_spectrum = spectral.PowerLaw(normalization = 1 * spectral.DIFFUSE_FLUX_UNIT, index = -2.7, e_ref =1* u.TeV)

#target_spectrum = spectral.PDG_ALL_PARTICLE
#simulated_spectrum = spectral.PowerLaw.from_simulation(simulated_events, obstime=0.1*u.s)

pyirf_weighted = [spectral.calculate_event_weights(energy*u.TeV, target_spectrum=target_spectrum, simulated_spectrum=simulated_spectrum) for energy in energies]
print(f"protons weighted pyirf = {sum(pyirf_weighted)}")

In [ ]:
sorted(pyirf_weighted, reverse=True)

In [ ]:
#pyirf_weighted = np.array(pyirf_weighted)*1000
histplot(pyirf_weighted, log_scale = True, bins = 25)
plt.xlabel("Energy in GeV")
plt.title("Muon Energies")

In [ ]:
#
#
# Images for events without cleaning
#
#

x_range_1 = [i for i in range(len(opt_eff_m))]
#std_arr = [np.std(opt_eff_m)/np.mean(opt_eff_m) for i,j in enumerate(opt_eff_m)]
std_arr_1 = [np.std(opt_eff_m)/i**0.5 for i,j in enumerate(opt_eff_m)]
std_arr_1_big = [np.std(opt_eff_m) for i,j in enumerate(opt_eff_m)]
#std_arr = [np.std(opt_eff) for i,j in enumerate(opt_eff)]


mean_range_1 = [i-1 for i in range(len(opt_eff_m)+3)]
y_upper_1 = [np.mean(opt_eff_m)+np.std(opt_eff_m) for i in mean_range_1]
y_lower_1 = [np.mean(opt_eff_m)-np.std(opt_eff_m) for i in mean_range_1]

y_mid_1 = [np.mean(opt_eff_m) for i in mean_range_1]
y_mid_1 = [np.mean(opt_eff_m[:i+1]) for i in range(len(opt_eff_m[:i+1]))]


#y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate(y_mid_1)]
#y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate(y_mid_1)]

y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate(y_mid_1)]
y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate(y_mid_1)]


#y_upper_1 = [j+np.std(opt_eff_m[:i+1]) for i,j in enumerate(y_mid_1)]
#y_lower_1 = [j-np.std(opt_eff_m[:i+1]) for i,j in enumerate(y_mid_1)]


y_upper_1 = [j+std_arr_1[-1] for i,j in enumerate([np.mean(opt_eff_m) for i in x_range_1])]
y_lower_1 = [j-std_arr_1[-1] for i,j in enumerate([np.mean(opt_eff_m) for i in x_range_1])]

plt.figure(figsize = (12,6))
#plt.ylim(0.095,0.34)

plt.style.use('default')
plt.xlim(-1,len(opt_eff_m))
plt.xlabel("Number of Events")
#plt.ylabel("Optical eff with rel. error for 1 measurment")
plt.ylabel("Optical efficiency")
#plt.title("Opt eff for muon rings without performing additional cleaning int_cut<0.1")
plt.title("Optical Efficiency from pure muonic simulation")
plt.grid(alpha=0.5)
plt.ylim(0,0.6)

#plt.scatter(x_range_1, opt_eff_m)
plt.errorbar(x_range_1, opt_eff_m, yerr=std_arr_1, fmt='o', ms = 7, alpha = 0.8, color = 'darkorange')#, c = 'blue')#, s = 100)#, c= 'black')


#plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'green', alpha = 0.3, label = 'mean +/- $1\sigma$')
plt.fill_between(x_range_1, y_upper_1, y_lower_1, color = 'green', alpha = 0.3, label = 'mean +/- $1\sigma$')
#plt.plot(x_range_1, y_mid_1, label = "mean", c = 'green', linewidth = 1.5)
plt.plot(x_range_1, [np.mean(opt_eff_m) for i in x_range_1], label = "mean", c = 'green', linewidth = 2.5)


plt.legend()
print("mean = ", np.mean(opt_eff_m))
print("std = ", np.std(opt_eff_m))
print("rel_err = ", np.std(opt_eff_m)/np.mean(opt_eff_m))
print("len = ", len(opt_eff_m))
#plt.savefig("/Users/vdk/ImagePurgatorium/4LSTMuons_RelStd.png")

In [ ]:
#
# DISPLAY IMAGES
#

int_cut = 0.3
min_pixel = 70

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()

for k in range(1,101):
    filename = f'/Users/vdk/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    print(filename)
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    checker = 0
    for i,j in enumerate(event_iterator):
        for m in j.trigger.tels_with_trigger:
            event = j
            checker += 1
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config_2)
            #muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            calib(j)
            image_processor(j)
            muon_processor(j)
            plt.figure(figsize = (9,6))
            camgeom = source.subarray.tel[m].camera.geometry
            title=f"event_{i}_telescope_{m}"
            #title=f"CleanedRing_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
            disp = CameraDisplay(camgeom,title=title)
            #disp.image = event_container[i].simulation.tel[m].true_image
            disp.image = j.r1.tel[m].waveform.sum(axis=1)
            disp.cmap = plt.cm.RdBu_r
            disp.add_colorbar()
            disp.set_limits_percent(95)
            reg_event_number.append(i)
            plt.show()
            plt.close()
    t2 = time.perf_counter(), time.process_time()
    print("checker = ", checker)   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
len(event_container)

In [ ]:
filename = f'/Users/vdk/4LSTmuons/simtel_run101_muon.simtel.gz'
print(filename)
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)
event = next(event_iterator)

In [ ]:
# PROCESSING OF REGULAR IMAGES
lets_check = 0
opt_eff_cleaned = []
reg_event_number = []
for i in event_container:
    for m in event_container[i].trigger.tel.keys():
        if not np.isnan(event_container[i].muon.tel[m].efficiency.width):
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                #plt.savefig(f"/Users/vdk/MRing_event{i}_fit{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}.png")
                
                print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("MSE ", event_container[i].muon.tel[m].parameters.mean_squared_error)                
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                #if event_container[i].dl1.tel[m].parameters.morphology.n_pixels > 50 and abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.1:
                #if abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1) < 0.2:
                #print("n_pixels ", event_container[i].dl1.tel[m].parameters.morphology.n_pixels)
                #print("Muon parameters ", event_container[i].muon.tel[m].parameters)   
                #print("intensity_check ", abs(event_container[i].muon.tel[m].parameters.intensity_ratio - 1))
                lets_check+=1
                plt.figure(figsize = (9,6))
                camgeom = source.subarray.tel[m].camera.geometry
                title=f"Event_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggercount{len(event_container[i].trigger.tel.keys())}"
                #title=f"CleanedRing_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
                disp = CameraDisplay(camgeom,title=title)
                disp.image = event_container[i].simulation.tel[m].true_image
                #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
                disp.cmap = plt.cm.RdBu_r
                disp.add_colorbar()
                disp.set_limits_percent(95)
                reg_event_number.append(i)
                plt.show()
                #plt.savefig(f"/Users/vdk/ImagePurgatorium/MuonsWithoutCleaning_intensitycut<0.3_event{i}.png")
                #opt_eff_cleaned.append(event_container[i].muon.tel[m].efficiency.optical_efficiency)
                    
plt.close()

In [ ]:
for m in event.trigger.tels_with_trigger:
    print(m)

In [ ]:
int_cut = 0.3
min_pixel = 70

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()

for k in range(1,101):
    filename = f'/Users/vdk/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    print(filename)
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    keks = k*1000
    checker = 0
    for i,j in enumerate(event_iterator):
        if len(j.trigger.tels_with_trigger) > 1:
            for m in j.trigger.tels_with_trigger:
                plt.figure(figsize = (9,6))
                camgeom = source.subarray.tel[m].camera.geometry
                title=f"event_{i}_telescope_{m}"
                #title=f"CleanedRing_{i}_optEff{round(event_container[i].muon.tel[m].efficiency.optical_efficiency,3)}_width{round(event_container[i].muon.tel[m].efficiency.width.to_value(),3)}_triggeredTel{m}"
                disp = CameraDisplay(camgeom,title=title)
                #disp.image = event_container[i].simulation.tel[m].true_image
                disp.image = j.r1.tel[m].waveform.sum(axis=1)
                disp.cmap = plt.cm.RdBu_r
                disp.add_colorbar()
                disp.set_limits_percent(95)
                reg_event_number.append(i)
                plt.show()
                plt.close()
            

In [ ]:
opt_eff = []

for i in event_container:
    for m in event_container[i].trigger.tels_with_trigger:
        opt_eff.append(event_container[i].muon.tel[m].efficiency.optical_efficiency)

In [ ]:
x_values = [i for i in range(len(event_container))]
plt.scatter(x_values, opt_eff)

In [ ]:
np.mean(opt_eff)

In [ ]:
len(opt_eff)

In [ ]:
from datetime import datetime

date_string = "21 June, 2018"

print("date_string =", date_string)
print("type of date_string =", type(date_string))

date_object = datetime.strptime(date_string, "%d %B, %Y")

print("date_object =", date_object)
print("type of date_object =", type(date_object))

In [ ]:
testtime = event.trigger.tel[1].time.to_value("mjd", "str")

In [ ]:
date_string = testtime

print("date_string =", date_string)
print("type of date_string =", type(date_string))

date_object = datetime.strptime(date_string, "%d %B, %Y")

print("date_object =", date_object)
print("type of date_object =", type(date_object))

In [ ]:
np.count_nonzero(event.simulation.tel[4].true_image)

In [ ]:
filename = f'/Users/vdk/LST/4LSTmuons/simtel_run101_muon.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)


In [ ]:
event = next(event_iterator)

In [ ]:
print(np.max(event.r1.tel[3].waveform))

In [ ]:
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
calib(event)
image_processor(event)
muon_processor(event)

In [ ]:
print(event.r0.tel[4].waveform[1])

In [ ]:
source.subarray.peek()

In [ ]:
import pickle

In [ ]:
with open("/Users/vdk/pickle_dump_4LSTstereoMuons.pkl", "rb") as input_file:
    events = pickle.load(input_file)

In [ ]:
for event in events:
    print(event)

In [ ]:
events.r1.tel[1].waveform

In [ ]:

plt.figure(figsize = (18,12))
camgeom = source.subarray.tel[1].camera.geometry
title = f'1'
disp = CameraDisplay(camgeom,title=title)
#disp.highlight_pixels(pixel_mask)
disp.image = events.r1.tel[4].waveform.sum(axis = 1)
disp.cmap = plt.cm.Reds
disp.add_colorbar()
disp.set_limits_percent(95)
#disp.set_limits_minmax(0,20)
plt.show()

In [ ]:
for tel in events.trigger.tels_with_trigger:
    print(tel)

In [ ]:
event.index.event_id

## Prod6 FIXCHI200 production

In [ ]:
int_cut = 0.3
min_pixel = 70
completeness = 0.9
containment = 0.5

muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", f"parameters.containment > {containment}"],
                                                        ["ring_completeness", f"parameters.completeness > {completeness}"]]},
                                    "ImageParameterQuery" : {
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]},
                                    "MuonIntensityFitter" : {
                                        "hole_radius_m" : [["type", "LST_*", 0.308]]
                                    }}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()
event_ids = []
impacts = []
widths = []


filename = f'/Users/vdk/UnrealLST/simtel_investigations/run901_muon.simtel.gz' # mono stereo
filename = f'/Users/vdk/UnrealLST/simtel_investigations/run1001_muon.simtel.gz' # stereo stereo

print(filename)
source = EventSource(filename)
event_iterator = iter(source)
checker = 0
for i,event in enumerate(event_iterator):
    for m in event.trigger.tels_with_trigger:
        checker += 1
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
        
        #print(f"Non Muon Event Telescope with stereo trigger = {event.trigger.tels_with_trigger}")
        calib(event)
        image_processor(event)
        muon_processor(event)
        for tel in event.trigger.tels_with_trigger:
            if not np.isnan(event.muon.tel[tel].efficiency.width):
                plt.figure(figsize = (9,6))
                camgeom = source.subarray.tel[1].camera.geometry
                title = f'event {event.index.event_id} in telescope {tel} and energy = {event.shower.simulation.energy}'
                disp = CameraDisplay(camgeom,title=title)
                disp.image = event.dl1.tel[tel].image
                disp.cmap = plt.cm.Reds
                disp.add_colorbar()
                disp.set_limits_percent(95)
                plt.show()
                print(f"Muon Event Telescope with stereo trigger = {event.trigger.tels_with_trigger}")
                event_container[i] = event
                event_ids.append(event.index.event_id)
                impacts.append(event.muon.tel[tel].efficiency.impact)
                widths.append(event.muon.tel[tel].efficiency.width)
t2 = time.perf_counter(), time.process_time()
print("checker = ", checker)   
plt.close()
print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
event_container.keys()

In [ ]:
event_container[83].trigger.tels_with_trigger

In [ ]:
print(event_container[83].simulation.shower)

In [ ]:
print(event_container[83].muon.tel[3].efficiency)

In [ ]:
print(event_container[83].muon.tel[4].efficiency)

In [ ]:
event

In [ ]:
print(event.pointing)

In [ ]:
print(event.trigger.tel[3])

In [ ]:
source.subarray.tel[1]

In [ ]:
print(source.subarray.tel[1].camera.geometry)

In [ ]:
print(source.subarray.tel)

In [ ]:
source.subarray.tels

In [ ]:
int_cut = 0.3
min_pixel = 70

# MuonIntensityFitter : 
#   hole_radius_m : [["type", "LST_*", 0.308]]

# muon_processor_config = Config({"MuonProcessor": 
#                                 {"RingQuery": 
#                                  {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
#                                                         ["ring_containment", "parameters.containment > 0.5"],
#                                                         ["ring_completeness", "parameters.completeness > 0.5"]]}}})

# #muon_processor_config = Config({"MuonProcessor":
# #                                {"ImageParameterQuery" :
# #                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
# #                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

# muon_processor_config_2 = Config({"MuonProcessor":
#                                 {"ImageParameterQuery" :
#                                  {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                         ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})


muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.3"],
                                                        ["ring_completeness", "parameters.completeness > 0.3"]]},
                                    "ImageParameterQuery" : {
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]},
                                    "MuonIntensityFitter" : {
                                        "hole_radius_m" : [["type", "MST_*", 0.108]]
                                    }}})

checker = 0
event_container = {}
t1 = time.perf_counter(), time.process_time()
event_ids = []
impacts = []
widths = []
filename = f'/Users/vdk/Software/ctasoft/calibpipe/calibpipe/tests/data/throughput/run101_muon_mst.simtel.gz'
print(filename)
source = EventSource(filename)
event_iterator = iter(source)
checker = 0
for i,event in enumerate(event_iterator):
    for m in event.trigger.tels_with_trigger:
        print(event.trigger.tels_with_trigger)
        checker += 1
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
        #muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
        #print(f"Non Muon Event Telescope with stereo trigger = {event.trigger.tels_with_trigger}")
        calib(event)
        image_processor(event)
        muon_processor(event)
        plt.figure(figsize = (9,6))
        camgeom = source.subarray.tel[1].camera.geometry
        title = f'event {event.index.event_id} in telescope {m}'
        disp = CameraDisplay(camgeom,title=title)
        disp.image = event.r1.tel[1].waveform.sum(axis = 1)
        # disp.image = event.dl1.tel[m].image
        # disp.cmap = plt.cm.Reds
        # disp.add_colorbar()
        # disp.set_limits_percent(95)
        plt.show()
        for m in event.muon.tel.keys():
            if not np.isnan(event.muon.tel[m].efficiency.width):
                #print("I am here")
                plt.figure(figsize = (9,6))
                camgeom = source.subarray.tel[1].camera.geometry
                title = f'event {event.index.event_id} in telescope {m}'
                disp = CameraDisplay(camgeom,title=title)
                disp.image = event.r1.tel[m].waveform.sum(axis = 1)
                # disp.image = event.dl1.tel[m].image
                # disp.cmap = plt.cm.Reds
                # disp.add_colorbar()
                # disp.set_limits_percent(95)
                plt.show()
                #print(f"Muon Event Telescope with stereo trigger = {event.trigger.tels_with_trigger}")
                event_container[keks+i] = event
                event_ids.append(event.index.event_id)
                impacts.append(event.muon.tel[m].efficiency.impact)
                widths.append(event.muon.tel[m].efficiency.width)
t2 = time.perf_counter(), time.process_time()
print("checker = ", checker)   
#plt.close()
print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
source.subarray.positions

In [ ]:
source.subarray.__dir__()

In [ ]:
event.trigger